In [23]:
import os
import time
import numpy as np
import cv2
from model.yolo_model import YOLO

In [24]:
def process_image(img):
    image = cv2.resize(img, (416, 416), interpolation=cv2.INTER_CUBIC) # interpolation=cv2.INTER_LINEAR for faster prediction but lower accuracy
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)
    return image

In [25]:
def get_classes(file):
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [26]:
def draw(image, boxes, scores, classes, all_classes):
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box
        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))
        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score), (top, left - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, cv2.LINE_AA)
        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))
    return image

In [27]:
def detect_image(image, yolo, all_classes):
    pimage = process_image(image)
    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()
    print('time: {0:.2f}s'.format(end - start))
    if boxes is not None:
        image = draw(image, boxes, scores, classes, all_classes)
    return image

In [28]:
def detect_video(video, yolo, all_classes):
    video_path = os.path.join('videos', video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow('video', cv2.WINDOW_NORMAL)
    while True:
        res, frame = camera.read()
        if not res:
            break
        image = detect_image(frame, yolo, all_classes)
        cv2.imshow('video', image)
        if cv2.waitKey(110) & 0xff == 27:
            break
    camera.release()

In [29]:
yolo = YOLO(0.6, 0.5)  # Removed the third argument './yolo.h5'
file = './coco_classes.txt'
all_classes = get_classes(file)

In [34]:
f = 'person.jpg'  # The image file name
path = os.path.join('images', f)  # Ensure correct path

# Read the image
image = cv2.imread(path)
if image is None:
    print(f"Error: Could not load image from {path}")
else:
    # Process and save the result if the image is loaded successfully
    image = detect_image(image, yolo, all_classes)
    cv2.imwrite(os.path.join('images/res/', f), image)


1/1 [==============================] - 0s 310ms/step
time: 0.35s
class: person, score: 1.00
box coordinate x,y,w,h: [ 180.20764589  332.23156548  744.65316474 1995.78970337]
class: person, score: 1.00
box coordinate x,y,w,h: [1060.23975849  463.46504736  372.34283924 1101.09101009]
